# Tutoriel Source : https://towardsdatascience.com/train-gpt-2-in-your-own-language-fc6ad4d60171

## Preparation

In [1]:
import os
from pathlib import Path
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFKC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(trainer, paths)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [2]:
paths = [str(x) for x in Path("../dataset/transformers_train/cult/").glob("**/*.txt")]

In [3]:
tokenizer = BPE_token()

tokenizer.bpe_train(paths)

save_path = "../dataset/transformers_train/tokenized_data/cult/"
tokenizer.save_tokenizer(save_path)

## Initialisation du model

In [4]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)

model = TFGPT2LMHeadModel(config)

In [6]:
single_string = ''
for filename in paths:
    with open(filename, "r", encoding='utf-8') as f:
        x = f.read()
    single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [7]:
examples = []
block_size = 50
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
    examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []

for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Train model

In [8]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [9]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/10


/home/juba/anaconda3/envs/nlp/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


5773/5773 [==============================] - 2565s 444ms/step - loss: 6.2072 - output_1_loss: 6.2072 - output_1_accuracy: 0.1448 - output_2_1_accuracy: 0.0017 - output_2_2_accuracy: 0.0014 - output_2_3_accuracy: 0.0017 - output_2_4_accuracy: 0.0016 - output_2_5_accuracy: 0.0015 - output_2_6_accuracy: 0.0018 - output_2_7_accuracy: 0.0021 - output_2_8_accuracy: 0.0018 - output_2_9_accuracy: 0.0016 - output_2_10_accuracy: 0.0019 - output_2_11_accuracy: 0.0018 - output_2_12_accuracy: 0.0018
Epoch 2/10
5773/5773 [==============================] - 2576s 446ms/step - loss: 5.4637 - output_1_loss: 5.4637 - output_1_accuracy: 0.1865 - output_2_1_accuracy: 0.0016 - output_2_2_accuracy: 0.0015 - output_2_3_accuracy: 0.0014 - output_2_4_accuracy: 0.0017 - output_2_5_accuracy: 0.0013 - output_2_6_accuracy: 0.0017 - output_2_7_accuracy: 0.0016 - output_2_8_accuracy: 0.0017 - output_2_9_accuracy: 0.0016 - output_2_10_accuracy: 0.0017 - output_2_11_accuracy: 0.0014 - output_2_12_accuracy: 0.0018
Epoch

In [10]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = './model_gpt2_cult/'
# creating directory if it is not present
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

('./model_gpt2_cult/tokenizer_config.json',
 './model_gpt2_cult/special_tokens_map.json',
 './model_gpt2_cult/vocab.json',
 './model_gpt2_cult/merges.txt',
 './model_gpt2_cult/added_tokens.json')